In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install transformers
# !pip3 install diffusers
!pip3 install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('/content/train.csv')

# Initialize lists
prompts = []
selected = []
rejected = []

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    prompt = row['prompt']
    s_completion = row['s_completion']
    n_completion = row['n_completion']

    prompts.append(prompt)
    selected.append(s_completion)
    rejected.append(n_completion)


In [ ]:
from datasets import Dataset

# Create a dictionary in the desired format
dpo_dataset_dict = {
    "prompt": prompts,
    "chosen": selected,
    "rejected": rejected,
}
del df
dataset = Dataset.from_dict(dpo_dataset_dict)

In [ ]:
from trl import DPOTrainer

from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

# dpo_trainer = DPOTrainer(
#      model,
#     #  model_ref,
#     #  args=training_args,
#      beta=0.1,
#      train_dataset=dataset,
#      tokenizer=tokenizer,
#  )

# dpo_trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments

In [ ]:
model_ref = model

In [ ]:
# training_args = TrainingArguments(
#     output_dir="./output",
#     num_train_epochs=1,
#     # auto_find_batch_size=4,
#     per_device_train_batch_size=4,
# #     gradient_accumulation_steps=2,
#     remove_unused_columns=False
# )

dpo_trainer = DPOTrainer(
     model,
     model_ref,
     args=training_args,
     beta=0.5,
     train_dataset=dataset,
     max_length = 100,
     tokenizer=tokenizer
 )

dpo_trainer.train()

In [ ]:
dpo_trainer.save_model("new-mistral")

In [ ]:
!zip -r new_gpt.zip /content/new-mistral

In [ ]:
from google.colab import files
files.download('new-mistral.zip')